# Imports

In [1]:
import numpy as np

# Generate data

In [2]:
# Seed
np.random.seed(0)

# Size
n_rows = 1000
n_cols = 40

# Data
X    = np.random.randn(n_rows, n_cols)
beta = np.random.randn(n_cols, 1)
Y    = X.dot(beta)

# Covariance
XTX = X.T.dot(X)
XTY = X.T.dot(Y)

# Gradient descent algorithm

In [3]:
def solveNQP(Q, q, epsilon, max_n_iter):
    
    # Initialize
    x          = np.zeros((n_cols, 1))
    x_diff     = np.zeros((n_cols, 1))
    grad_f     = q.copy()
    grad_f_bar = q.copy()

    # Loop over iterations
    for i in range(max_n_iter):
    
        # Get passive set information
        passive_set = np.logical_or(x > 0, grad_f < 0)
        n_passive = np.sum(passive_set)

        # Calculate gradient
        grad_f_bar[:] = grad_f
        grad_f_bar[~passive_set] = 0
        grad_norm = np.vdot(grad_f_bar, grad_f_bar)
        
        # Abort?
        if (n_passive == 0 or grad_norm < epsilon):
            break
            
        # Exact line search
        alpha_den = np.vdot(grad_f_bar, Q.dot(grad_f_bar))
        alpha = grad_norm / alpha_den
            
        # Update x
        x_diff = -x.copy()
        x -= alpha * grad_f_bar
        x = np.maximum(x, 0.)
        x_diff += x
    
        # Update gradient
        grad_f += Q.dot(x_diff)
        
    # Return
    return x


def NNLS_GD(XTX, XTY, epsilon=1e-10, max_n_iter=100):

    # Normalization factors
    H_diag = np.diag(XTX).copy().reshape(-1,1)
    Q_den  = np.sqrt(np.outer(H_diag, H_diag))
    q_den  = np.sqrt(H_diag)

    # Normalize
    Q =  XTX / Q_den
    q = -XTY.reshape(-1,1) / q_den
    
    # Solve NQP
    y = solveNQP(Q, q, epsilon, max_n_iter)

    # Undo normalization
    x = y / q_den

    # Return
    return x

# Coordinate descent algorithm

In [4]:
def NNLS_CD(XTX, XTY, epsilon=1e-10, max_n_iter=100):
    
    # Initialize
    x  = np.zeros((n_cols, 1))
    f  = -XTY.reshape(-1,1)
    mu = f.copy()
    
    # Loop over iterations
    for i in range(max_n_iter):
        
        # Abort?
        Hxf = XTX.dot(x) + f
        criterion_1 = np.all(Hxf >= -epsilon)
        criterion_2 = np.all(Hxf[x > 0] <= epsilon)
        if (criterion_1 and criterion_2):
            break
    
        # Loop over coordinates
        for k in range(n_cols):
            x_diff  = -x[k,0]
            x[k,0]  = np.maximum(0., x[k,0] - mu[k,0] / XTX[k,k])
            x_diff += x[k,0]
            mu     += x_diff * XTX[:,k].reshape(-1,1)
            
    # Return
    return x

# Timing comparison
- Nested python for-loop in CD is very inefficient
- Need to port to C for a real test

In [5]:
print ("Gradient descent algorithm:")
%timeit NNLS_GD(XTX, XTY)
print ("\n")
print ("Coordinate descent algorithm:")
%timeit NNLS_CD(XTX, XTY)

Gradient descent algorithm:
1000 loops, best of 3: 263 µs per loop


Coordinate descent algorithm:
100 loops, best of 3: 2.79 ms per loop


# Objective comparison

In [6]:
def objective(X, Y, beta_hat):
    assert np.all(beta_hat >= 0)
    err = X.dot(beta_hat.reshape(-1,1)) - Y.reshape(-1,1)
    return np.vdot(err, err)

beta_hat_gd = NNLS_GD(XTX, XTY)
beta_hat_cd = NNLS_GD(XTX, XTY)

print (objective(X, Y, beta_hat_gd))
print (objective(X, Y, beta_hat_cd))

25767.4294909
25767.4294909
